In [1]:
from scipy import stats

In [9]:
import numbers
from scipy.optimize import minimize_scalar
from scipy.special import gammaln
from scipy.integrate import quad

class KernelDensityEstimator(object):
    def __init__(self, X, bandwidth_method=None, domain=None):
        # Store all the values
        self.X = np.asarray(X)
        self.n = len(self.X)
        self.bandwidth_method = bandwidth_method
        # Store the domain
        if domain is None:
            self.domain = (-np.inf, np.inf)
        else:
            self.domain = domain
        
        # Use cross-validation if desired
        if bandwidth_method=='cross_validation':
            self.bandwidth = self.cross_validation_optimisation()
        elif isinstance(self.bandwidth_method, numbers.Number):
            self.bandwidth = self.bandwidth_method
            self.bandwidth_method = 'numeric'
            
    def __call__(self, x, bandwidth=None):
        return self.evaluate(x, bandwidth)
        
    def evaluate_kernel(self, bandwidth, X, x):
        raise NotImplementedError
        
    def evaluate_kernel_product_integral(self, bandwidth, X1, X2):
        assert len(X1) == self.n and len(X2) == self.n, "The length of the data vectors must be identical."
        # Create a container
        contributions = np.zeros((self.n, self.n))
        # Iterate over all combinations
        for i in range(self.n):
            for j in range(i, self.n):
                integrand = lambda x: self.evaluate_kernel(bandwidth, X1[i], x) \
                    * self.evaluate_kernel(bandwidth, X2[j], x)
                integral = quad(integrand, *self.domain)
                contributions[i, j] = contributions[j, i] = integral[0]
                
        return contributions
        
    def evaluate(self, x, bandwidth=None):
        # Evaluate the density at all points
        contributions = self.evaluate_kernel(bandwidth or self.bandwidth, 
                                             self.X, x)
        # Sum up the contributions
        return np.mean(contributions, axis=1)
    
    def cross_validation_score(self, bandwidth):
        # Return something unpleasant for negative bandwidths
        if bandwidth <= 0:
            return np.inf
        try:
            # Evaluate the integral of the product of the kernels for all points
            contributions = self.evaluate_kernel_product_integral(bandwidth, self.X, self.X)
            score = np.mean(contributions)
        except NotImplementedError:
            raise NotImplementedError('`evaluate_kernel_product_integral` must be implemented for cross validation.')
        # Evaluate the estimate at all data points
        contributions = self.evaluate_kernel(bandwidth, self.X, self.X)
        contributions[range(self.n), range(self.n)] = 0
        # Obtain the part due to the linear terms in (3.35)
        score -= 2 * np.sum(contributions) / (self.n * (self.n - 1))
        return score
    
    def cross_validation_optimisation(self):
        return minimize_scalar(self.cross_validation_score)['x']
    
class GaussianEstimator(KernelDensityEstimator):
    def __init__(self, X, bandwidth_method='silverman'):
        super(GaussianEstimator, self).__init__(X, bandwidth_method)
        
        if bandwidth_method=='silverman':
            self.bandwidth = self.silverman()
            
    def silverman(self):
        return 1.06 * np.std(self.X) * self.n ** -0.2
        
    def evaluate_kernel(self, bandwidth, X, x):
        # Evaluate the exponent of the gaussian kernel
        chi2 = (X[None, :] - x[:, None]) ** 2
        # Exponentiate and apply normalisation
        return np.exp(-.5 * chi2 / bandwidth**2) \
            / (np.sqrt(2 * np.pi) * bandwidth)
        
    def evaluate_kernel_product_integral(self, bandwidth, X1, X2):
        # This is much easier for a Gaussian kernel. No numerical integration needed.
        return self.evaluate_kernel(np.sqrt(2) * bandwidth, X1, X2)
    
class VariableGaussianEstimator(KernelDensityEstimator):
    def __init__(self, X, bandwidth_function, shift_function, bandwidth_method=None):
        super(VariableGaussianEstimator, self).__init__(X, bandwidth_method, (0, np.inf))
        self.bandwidth_function = bandwidth_function
        self.shift_function = shift_function
        
    def evaluate_kernel(self, bandwidth, X, x):
        # Evaluate the shift and variable bandwidth
        shift = self.shift_function(bandwidth, X, x)
        bandwidth = self.bandwidth_function(bandwidth, X, x)
        # Evaluate the chi-square
        chi2 = (X[None, :] - x[:, None] - shift) ** 2
        return np.exp(-.5 * chi2 / bandwidth**2) \
            / (np.sqrt(2 * np.pi) * bandwidth)
    
class ImproperGammaEstimator(KernelDensityEstimator):
    def __init__(self, X, bandwidth_method='plugin'):
        super(ImproperGammaEstimator, self).__init__(X, bandwidth_method)
        
        if bandwidth_method=='plugin':
            self.bandwidth = self.plugin()
            
    def to_variable_gaussian(self):
        # Convert to a variable Gaussian estimator
        shift_function = lambda bandwidth, _, _2: bandwidth * bandwidth
        bandwidth_function = lambda bandwidth, _, x: bandwidth * np.sqrt(bandwidth * bandwidth + x[:, None])
        variable = VariableGaussianEstimator(self.X, bandwidth_function, shift_function, self.bandwidth)
        return variable
            
    def evaluate_kernel(self, bandwidth, X, x):
        # Calculate the shape and scale parameters
        theta = bandwidth ** 2
        if isinstance(x, np.ndarray):
            k = 1 + x[:, None] / theta
        else:
            k = 1 + x / theta
        # Evaluate the gamma distribution
        if isinstance(X, np.ndarray):
            X = X[None, :]
        loggamma = (k-1) * np.log(X) - X / theta - k * np.log(theta) - gammaln(k)
        return np.exp(loggamma)
            
    def plugin(self):
        # Compute the logarithmic mean and variance
        X = np.log(self.X)
        mu = np.mean(X)
        Sigma = np.std(X)
        
        return (2**0.8*np.exp(mu/2.)*Sigma)/(np.exp((17*Sigma**2)/8.)*self.n*(12 + 20*Sigma**2 + 9*Sigma**4))**0.2
    
    def plugin_score(self, bandwidth):
        X = np.log(self.X)
        mu = np.mean(X)
        Sigma = np.std(X)
        
        return np.exp(-3*mu + Sigma**2/8.) \
                *((64*np.exp((5*mu)/2.))/self.n + 
                  (np.exp((17*Sigma**2)/8.)* bandwidth**5*(12 + 20*Sigma**2 + 9*Sigma**4))/Sigma**5) \
            /(128.*np.sqrt(np.pi)*bandwidth)
            
class ProperGammaEstimator(KernelDensityEstimator):
    def __init__(self, X, bandwidth_method='plugin'):
        super(ProperGammaEstimator, self).__init__(X, bandwidth_method)
        
        if bandwidth_method=='plugin':
            self.bandwidth = self.plugin()
            
    def to_variable_gaussian(self):
        # Convert to a variable Gaussian estimator
        shift_function = lambda bandwidth, _, _2: bandwidth * bandwidth
        bandwidth_function = lambda bandwidth, _, x: bandwidth * np.sqrt(bandwidth * bandwidth + X[None, :])
        variable = VariableGaussianEstimator(self.X, bandwidth_function, shift_function, self.bandwidth)
        return variable
            
    def evaluate_kernel(self, bandwidth, X, x):
        # Calculate the shape and scale parameters
        theta = bandwidth ** 2
        if isinstance(X, np.ndarray):
            k = 1 + X[:, None] / theta
        else:
            k = 1 + X / theta
        # Evaluate the gamma distribution
        if isinstance(x, np.ndarray):
            x = x[None, :]
        loggamma = (k-1) * np.log(x) - x / theta - k * np.log(theta) - gammaln(k)
        return np.exp(loggamma)
    
    def evaluate_kernel_product_integral(self, bandwidth, X1, X2):
        # Calculate the shape and scale parameters
        theta = bandwidth ** 2
        k1 = 1 + X1[None, :] / theta
        k2 = 1 + X2[:, None] / theta
        # Evaluate
        loggamma = (1 - k1 - k2) * np.log(2) + gammaln(k1 + k2 - 1) \
            - gammaln(k1) - gammaln(k2)
        return np.exp(loggamma) / theta
            
    def plugin(self):
        # Compute the logarithmic mean and variance
        X = np.log(self.X)
        mu = np.mean(X)
        Sigma = np.std(X)
        
        return (2**0.8*np.exp(mu/2.)*Sigma)/(np.exp((17*Sigma**2)/8.)*self.n*(12 + 4*Sigma**2 + Sigma**4))**0.2
    
    def plugin_score(self, bandwidth):
        X = np.log(self.X)
        mu = np.mean(X)
        Sigma = np.std(X)
        
        return (np.exp(-3*mu + Sigma**2/8.)*((64*np.exp((5*mu)/2.))/self.n + 
           (np.exp((17*Sigma**2)/8.)*bandwidth**5*(12 + 4*Sigma**2 + Sigma**4))/Sigma**5))/(128.*np.sqrt(np.pi)*bandwidth)

# Simple Gaussian example

In [3]:
# Generate samples
distribution = stats.norm()
X = distribution.rvs(size=300)

# Fit a density estimator
kde = GaussianEstimator(X, 'cross_validation')

# Plot the original distribution and the KDE
x = np.linspace(-5, 5)
plt.plot(x, kde(x))
plt.plot(x, distribution.pdf(x))

# Simple example for improper Gamma KDE

In [6]:
# Generate samples
np.random.seed(123)
mu = 1
Sigma = 1
distribution = stats.lognorm(Sigma, scale=np.exp(mu))
X = distribution.rvs(size=300)
print np.mean(np.log(X)), np.std(np.log(X))

# Fit a density estimator
kde = ImproperGammaEstimator(X, 'plugin')

# Plot the original distribution and KDEs
x = np.linspace(1e-4, 15, 500)
plt.plot(x, kde(x), color='k', label='Improper Gamma estimator')
plt.plot(x, distribution.pdf(x), color='k', ls=':', label='Reference distribution')

plt.scatter(X, np.zeros_like(X), marker='|', color='k')

# Finally plot the approximation with a Gaussian
kde_balloon = kde.to_variable_gaussian()
plt.plot(x, kde_balloon.evaluate(x), color='k', ls='--', label='Gaussian approximation')

plt.xlim(-1,15)
plt.xlabel('Random variable $X$')
plt.ylabel('Density')
plt.legend(frameon=False, loc='best')
plt.tight_layout()
plt.savefig('../paper/improper-gamma.pdf')

0.97081406678 1.0265454538


# Comparison of LOO cross-validation and plugin scores 

In [25]:
# Define a reference distribution
distribution = stats.lognorm(Sigma, scale=np.exp(mu))
# Define a bandwidth range
bandwidths = np.logspace(-2, 0, 50)

# Define containers for the scores
plugin_scores=[]
cv_scores=[]

# Iterate and evaluate the bandwidths
runs = 1000
for run in range(runs):
    if (run + 1) % 20 == 0:
        print run + 1,
    # Generate data
    X = distribution.rvs(size=300)
    # Fit an estimator
    kde = ProperGammaEstimator(X, None)
    # Evaluate the scores of the quality function
    plugin_scores.append([kde.plugin_score(bw) for bw in bandwidths])
    cv_scores.append([kde.cross_validation_score(bw) for bw in bandwidths])

20 40 60 80 100 120 140 160 180 200 220 240 260 280 300 320 340 360 380 400 420 440 460 480 500 520 540 560 580 600 620 640 660 680 700 720 740 760 780 800 820 840 860 880 900 920 940 960 980 1000


In [26]:
def plot_scores(scores, f=5, color='k', ls='-', offset=0, label=None, ax=None):
    # Get default axes if none are given
    ax = ax or plt.gca()
    
    # Get the median and the interval
    scores = np.asarray(scores)
    median = np.median(scores, axis=0)
    # Adjust the offset
    offset -= np.min(median)
    median += offset
    scores += offset
    lower = np.percentile(scores, f, axis=0)
    upper = np.percentile(scores, 100-f, axis=0)
    
    # Plot
    ax.plot(bandwidths, median, color=color, ls=ls, label=label)
    ax.fill_between(bandwidths, lower, upper, alpha=.25, color=color)
    ax.scatter(bandwidths[np.argmin(median)], np.min(median), color=color)
    
plot_scores(plugin_scores, label='Plugin')
plot_scores(cv_scores, ls='--', offset=0.07, label='Cross-validation')
plt.xscale('log')
plt.xlim(0, 1)
plt.xlabel('Bandwidth $\sigma$')
plt.ylabel('MISE score (arbitrary offset)')
plt.legend(loc='best', frameon=False)

plt.tight_layout()
plt.savefig('../paper/bandwidth-comparison.pdf')

# Comparison of kernels

In [35]:
ts = 2 * np.arange(1, 5)
x = np.linspace(0, 5, 200)
    
for t in ts:
    k = t + 1
    theta = 1.0 / t
    y = stats.gamma.pdf(x, k, scale=theta)
    plt.plot(x, y, color='k')
    
    mu = k * theta
    sigma = np.sqrt(mu * theta)
    y = stats.norm.pdf(x, mu, sigma)
    plt.plot(x, y, color='r')